In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import RandomOverSampler

In [214]:
file = 'train_preprocessed_type_of_loan.csv'

df = pd.read_csv("../Data/" + file, low_memory=False)

In [13]:
from itertools import product
from msilib.schema import Condition

miss = [True, False]
sampling = [True, False]
aug_OH = [True, False]
not_spec = [True, False]
a = [miss, sampling, aug_OH, not_spec]
b = [miss]*4

conditions = [[0, 1]]*4
list(product(*conditions))

[(0, 0, 0, 0),
 (0, 0, 0, 1),
 (0, 0, 1, 0),
 (0, 0, 1, 1),
 (0, 1, 0, 0),
 (0, 1, 0, 1),
 (0, 1, 1, 0),
 (0, 1, 1, 1),
 (1, 0, 0, 0),
 (1, 0, 0, 1),
 (1, 0, 1, 0),
 (1, 0, 1, 1),
 (1, 1, 0, 0),
 (1, 1, 0, 1),
 (1, 1, 1, 0),
 (1, 1, 1, 1)]

In [218]:
file = 'train_preprocessed.csv'

In [219]:
if file == 'train_preprocessed.csv':
    features_int = list(df.select_dtypes(include=["number"]).columns[-10:])
    features_cat = list(df.select_dtypes(include=["object"]).columns)
    
    features = features_int + features_cat
elif file == 'train_preprocessed_type_of_loan.csv':
    features = list(df.select_dtypes(include=["object"]).columns)

for feature in features:
    df[feature] = df[feature].astype('category')

df['Credit_Score'] = df['Credit_Score'].astype('category')

In [ ]:
def features(df, Not_specified, A_OH):
    tmp = df.copy()
    if Not_specified:
        if A_OH:
            uniq_types = tmp['Type_of_Loan'].value_counts()[:10].index
            for uniq_type in uniq_types:
                tmp[uniq_type] = tmp['Type_of_Loan'].str.count(uniq_type)
            
            features_num = list(tmp.select_dtypes(include=["number"]).columns[-10:])
            features_cat = list(tmp.select_dtypes(include=["object"]).columns)
            
            features = features_num + features_cat
            for feature in features:
                tmp[feature] = tmp[feature].astype('category')
            
            tmp.drop(columns='Type_of_Loan', inplace=True)
        else:
            features = list(tmp.select_dtypes(include=["object"]).columns)
            
            for feature in features:
                tmp[feature] = tmp[feature].astype('category')
    else:
        tmp.loc[tmp['Type_of_Loan'].str.contains('Not Specified'), 'Type_of_Loan'] = None
        
        if A_OH:
            uniq_types = tmp['Type_of_Loan'].value_counts()[:9].index
            for uniq_type in uniq_types:
                tmp[uniq_type] = tmp['Type_of_Loan'].str.count(uniq_type)
            
            features_num = list(tmp.select_dtypes(include=["number"]).columns[-9:])
            features_cat = list(tmp.select_dtypes(include=["object"]).columns)
            
            features = features_num + features_cat
            for feature in features:
                tmp[feature] = tmp[feature].astype('category')
                
            tmp.drop(columns='Type_of_Loan', inplace=True)
        else:
            features = list(tmp.select_dtypes(include=["object"]).columns)
            
            for feature in features:
                tmp[feature] = tmp[feature].astype('category')
    
    return tmp   

In [14]:
def data_splitting(df, cat=cat, num=num, nan=False):
    if nan:
        tmp = df.copy()
    else:
        tmp = df.dropna().reset_index(drop=True).copy()
        
    return train_test_split(tmp[num + cat], tmp['Credit_Score'], test_size=0.2, shuffle=False)

def data_resampler(X_train, y_train, enabled=False):
    if enabled:
        sampler = RandomOverSampler()
        X_train, y_train = sampler.fit_resample(X_train, y_train) 
    else:
        pass
    return X_train, y_train   

NameError: name 'cat' is not defined

In [ ]:
def table(df, cat, num, model='SGD'):
    """m: miss
       s: sampling
       a: aug_OH
       n: not_spec

    Args:
        df (_type_): _description_
        model (str, optional): _description_. Defaults to 'SGD'.
    """
    from itertools import product   
    for m, s, a, n in product(conditions = [[0, 1]]*4):
        tmp = features(df, n, a)
        X_train, X_test, y_train, y_test = data_splitting(df, cat, num, m)
        X_train, y_train = data_resampler(X_train, y_train, s)
        
            
        
    
    
    

In [186]:
df.loc[df['Type_of_Loan'].str.contains('Not Specified'), 'Type_of_Loan'] = None

In [215]:
uniq_types = df['Type_of_Loan'].value_counts()[:10].index
uniq_types

Index(['no Loan', 'Not Specified', 'Credit-Builder Loan', 'Personal Loan',
       'Debt Consolidation Loan', 'Student Loan', 'Payday Loan',
       'Mortgage Loan', 'Auto Loan', 'Home Equity Loan'],
      dtype='object')

In [216]:
# "augmented" oneHotEncoding 
for uniq_type in uniq_types:
    df[uniq_type] = df['Type_of_Loan'].str.count(uniq_type)

In [217]:
df.drop(columns='Type_of_Loan', inplace=True)

In [220]:
num = df.drop(columns=['Credit_Score']).select_dtypes(include=['number']).columns.to_list()
cat = df.drop(columns=['Credit_Score']).select_dtypes(include=['category']).columns.to_list()

In [24]:
bol = 0
a, b = 10*bol + 1, 2*(1-bol)
a, b

(1, 2)

In [25]:
bol = 1
a, b = 10*bol + 1, 2*(1-bol)
a, b

(11, 0)

In [236]:
#X_train, X_test, y_train, y_test = train_test_split(df[num + cat], df['Credit_Score'], test_size=0.2)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

sampler = RandomOverSampler() 

X_train, X_test, y_train, y_test = data_splitting(df, cat, num, True)
#X_train, y_train = sampler.fit_resample(X_train, y_train)


cat_preprocessor = Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                            ("onehot", OneHotEncoder(sparse=True, handle_unknown="ignore"))])

num_preprocessor = Pipeline([
                            ("imputer", SimpleImputer(strategy="median"))])

preprocessor = ColumnTransformer([
    ("numerical", num_preprocessor, num),
    ("categorical", cat_preprocessor, cat)
])

## Transforming
X_train = preprocessor.fit_transform(X_train)#.todense()
X_test = preprocessor.transform(X_test)

scaler = MaxAbsScaler()
X_train = scaler.fit_transform(X_train)#.toarray()
X_test = scaler.transform(X_test)#.toarray()

In [237]:
X_train.shape

(80000, 103)

# SVM 

In [238]:
from sklearn.linear_model import SGDClassifier

svc = SGDClassifier(loss="hinge", n_jobs=10, validation_fraction=0.25)

svc.fit(X_train, y_train)

SGDClassifier(n_jobs=10, validation_fraction=0.25)

In [244]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

print(classification_report(svc.predict(X_train), y_train))

              precision    recall  f1-score   support

           0       0.58      0.61      0.59     22110
           1       0.64      0.74      0.69     37243
           2       0.74      0.51      0.60     20647

    accuracy                           0.64     80000
   macro avg       0.66      0.62      0.63     80000
weighted avg       0.65      0.64      0.64     80000



In [245]:
f1_score(svc.predict(X_train), y_train, average=None), accuracy_score(svc.predict(X_train), y_train)

(array([0.59401419, 0.68686184, 0.60379423]), 0.6424)

In [240]:
print(classification_report(svc.predict(X_test), y_test))

              precision    recall  f1-score   support

           0       0.58      0.61      0.60      5315
           1       0.65      0.75      0.70      9342
           2       0.78      0.54      0.64      5343

    accuracy                           0.66     20000
   macro avg       0.67      0.63      0.64     20000
weighted avg       0.67      0.66      0.65     20000



Type of Loan with 'Not Specified' values
| Model | Is NaN? | rng Sampling | Train | Test |
|:-----:|:-------:|:------------:|-------|------|
|  SGD  |    y    |       n      | 0.71  | 0.65 |
|  SGD  |    n    |       n      | 0.72  | 0.64 |
|  SGD  |    n    |       y      | 0.74  | 0.66 |
|  SGD  |    y    |       y      | 0.72  | 0.65 |

Type of Loan with 'Not Specified' values & Augmented OneHot
| Model | Is NaN? | rng Sampling | Train | Test |
|:-----:|:-------:|:------------:|-------|------|
|  SGD  |    y    |       n      | 0.64  | 0.66 |
|  SGD  |    n    |       n      | 0.64  | 0.65 |
|  SGD  |    n    |       y      | 0.70  | 0.67 |
|  SGD  |    y    |       y      | 0.69  | 0.66 |

Type of Loan without 'Not Specified' values
| Model | Is NaN? | rng Sampling | Train | Test |
|:-----:|:-------:|:------------:|-------|------|
|  SGD  |    y    |       n      | 0.68  | 0.65 |
|  SGD  |    n    |       n      | 0.71  | 0.64 |
|  SGD  |    n    |       y      | 0.73  | 0.66 |
|  SGD  |    y    |       y      | 0.70  | 0.66 |

Type of Loan without 'Not Specified' values & Augmented OneHot
| Model | Is NaN? | rng Sampling | Train | Test |
|:-----:|:-------:|:------------:|-------|------|
|  SGD  |    y    |       n      | 0.64  | 0.65 |
|  SGD  |    n    |       n      | 0.65  | 0.64 |
|  SGD  |    n    |       y      | 0.69  | 0.67 |
|  SGD  |    y    |       y      | 0.70  | 0.67 |